In [5]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain.vectorstores import Chroma


from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader



In [1]:

# Initialize text splitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Load documents
documents = [
    "This is document 1. It talks about machine learning.",
    "This is document 2. It focuses on retrieval-augmented generation.",
    "This is document 3. It covers generative AI and large language models."
]

# Split documents into chunks
texts = text_splitter.split_text(" ".join(documents))

# Initialize ChromaDB
embedding_model = OpenAIEmbeddings()  # Use OpenAI-compatible embeddings
chroma_db = Chroma.from_texts(texts, embedding_model, persist_directory="./chroma_db")

# Save ChromaDB
chroma_db.persist()


C:\Users\dell\AppData\Local\Temp\ipykernel_8228\1004005750.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding_model = OpenAIEmbeddings()  # Use OpenAI-compatible embeddings


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [6]:
from langchain.text_splitter import CharacterTextSplitter

# Sample documents
documents = [
    "Retrieval-Augmented Generation combines retrieval systems with generative AI models.",
    "Sentence Transformers are excellent for creating text embeddings.",
    "ChromaDB is a vector database that efficiently handles embeddings."
]

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=20)
chunks = []
for doc in documents:
    chunks.extend(text_splitter.split_text(doc))

print("Chunks:", chunks)


Chunks: ['Retrieval-Augmented Generation combines retrieval systems with generative AI models.', 'Sentence Transformers are excellent for creating text embeddings.', 'ChromaDB is a vector database that efficiently handles embeddings.']


In [7]:
from sentence_transformers import SentenceTransformer

# Load Sentence Transformers model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = embedding_model.encode(chunks)

print("Generated Embeddings Shape:", embeddings.shape)


Generated Embeddings Shape: (3, 384)


In [9]:
from chromadb import Client
from chromadb.config import Settings


# Initialize the ChromaDB client
client = Client()

# Create a collection for storing embeddings
collection = client.create_collection("rag_documents")

# Add chunks and embeddings to the collection
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),  # Convert NumPy array to list
    metadatas=[{"source": f"doc_{i}"} for i in range(len(chunks))],
    ids=[f"chunk_{i}" for i in range(len(chunks))]
)

print("Embeddings stored in ChromaDB.")


Embeddings stored in ChromaDB.


In [10]:
query = "What is Retrieval-Augmented Generation?"

# Generate query embedding
query_embedding = embedding_model.encode([query])[0]

# Perform similarity search
results = collection.query(query_embeddings=[query_embedding], n_results=2)

# Retrieve documents
print("Relevant Documents:", results["documents"])


Relevant Documents: [['Retrieval-Augmented Generation combines retrieval systems with generative AI models.', 'Sentence Transformers are excellent for creating text embeddings.']]


# error

In [16]:
from langchain.llms import LlamaCpp

# Load a local Llama model
llm = LlamaCpp(model_path=r"C:\Users\dell\.ollama\models\blobs\sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff")





llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from C:\Users\dell\.ollama\models\blobs\sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                               general.tags arr[s

In [19]:
import requests
import json

# Define a function to query the Ollama API
def query_ollama(prompt, model="llama3.2", api_url="http://localhost:11434/api/generate"):
    """
    Query the Ollama model using its local API.
    """
    headers = {'Content-Type': 'application/json'}
    data = {
        "model": model,
        "prompt": prompt
    }
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        result = response.json()
        return result.get('response', "No response found.")
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Test the LLM using the API
if __name__ == "__main__":
    # Combine the retrieved documents into a prompt

    prompt = f"HOw are you"

    response = query_ollama(prompt)
    print("Response from Llama 3.2:")
    print(response)


JSONDecodeError: Extra data: line 2 column 1 (char 93)

In [21]:
import requests
import json

# Function to query the Ollama API
def query_ollama(prompt, model="llama3.2", api_url="http://localhost:11434/api/generate"):
    """
    Query the Ollama model using its local API and handle possible streaming responses.
    """
    headers = {'Content-Type': 'application/json'}
    data = {
        "model": model,
        "prompt": prompt
    }

    # Send the request
    response = requests.post(api_url, headers=headers, data=json.dumps(data), stream=True)
    
    # Handle streaming response
    full_response = []
    try:
        for line in response.iter_lines():
            if line:  # Ignore keep-alive lines
                decoded_line = json.loads(line.decode('utf-8'))
                full_response.append(decoded_line.get('response', ''))
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        print(f"Problematic line: {line.decode('utf-8')}")
    finally:
        response.close()

    # Join the response into a single string
    return "".join(full_response)

# Example query
if __name__ == "__main__":
    # Simulate retrieval results
    results = {
        "documents": [
            [
                "RAG combines the strengths of retrieval-based methods and generative models.",
                "It retrieves relevant context from a database to enhance the generative model's output.",
                "This approach improves accuracy and relevance, especially for domain-specific queries."
            ]
        ]
    }
    query = "What is the benefit of using RAG for domain-specific queries?"
    
    # Combine the retrieved documents into a prompt
    context = "\n".join(results["documents"][0])
    prompt = f"Answer the following question based on the context:\n\nContext:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"
    
    # Query the Llama 3.2 model via Ollama API
    response = query_ollama(prompt)
    print("Response from Llama 3.2:")
    print(response)


Response from Llama 3.2:
The benefit of using RAG for domain-specific queries is that it improves accuracy and relevance.


# pdf based Rag

In [23]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import requests
import json


# Function to load and process PDF files
def load_pdf_files(pdf_dir):
    """
    Load and process all PDF files from a directory.
    Splits the text into chunks for embedding.
    """
    documents = []
    pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        print(f"Processing {pdf_file}...")
        loader = PyPDFLoader(pdf_file)
        pages = loader.load()
        documents.extend(pages)

    # Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    return chunks


# Function to store chunks in a vector database
def create_vector_db(chunks, db_path="./chroma_db"):
    """
    Create or update a vector database using Chroma.
    """
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Use Sentence Transformers
    chroma_db = Chroma.from_documents(chunks, embedding_model, persist_directory=db_path)
    chroma_db.persist()
    return chroma_db


# Function to query the Ollama API
def query_ollama(prompt, model="llama3.2", api_url="http://localhost:11434/api/generate"):
    """
    Query the Ollama model using its local API.
    """
    headers = {'Content-Type': 'application/json'}
    data = {"model": model, "prompt": prompt}

    response = requests.post(api_url, headers=headers, data=json.dumps(data), stream=True)

    # Handle streaming response
    full_response = []
    try:
        for line in response.iter_lines():
            if line:  # Ignore keep-alive lines
                decoded_line = json.loads(line.decode('utf-8'))
                full_response.append(decoded_line.get('response', ''))
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        print(f"Problematic line: {line.decode('utf-8')}")
    finally:
        response.close()

    return "".join(full_response)


# Main program
if __name__ == "__main__":
    # Directory containing PDF files
    pdf_directory = r"G:\7Th-Semester\information-Security\Pre-Mid"  # Replace with your PDF directory path

    # Load and process PDF files
    print("Loading and processing PDF files...")
    pdf_chunks = load_pdf_files(pdf_directory)

    # Create a vector database
    print("Creating vector database...")
    vector_db = create_vector_db(pdf_chunks)

    # Input query from the user
    query = input("Enter your query: ")

    # Retrieve relevant documents
    print("Retrieving relevant documents...")
    search_results = vector_db.similarity_search(query, k=3)  # Retrieve top 3 relevant chunks

    # Combine retrieved documents into a context
    context = "\n".join([doc.page_content for doc in search_results])
    prompt = f"Answer the following question based on the context:\n\nContext:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"

    # Query the Llama 3.2 model via Ollama API
    print("Querying Llama 3.2 model...")
    response = query_ollama(prompt)

    # Display the response
    print("\nResponse from Llama 3.2:")
    print(response)


Loading and processing PDF files...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 1.1 (Lab Environment Setup).pdf...


Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)


Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 1.2 (Recap of OS with Linux).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 1.3 (Recap of InterNetworking Concepts with Linux).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.1 (Ethical Hacking Pentesting and Anonymity).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.2 (Reconnaissance Info Gathering and OSINT) (1).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.3 (Scanning and Vulnerability Analysis - I).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.4 (Scanning and Vulnerability Analysis - II).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.5 (Exploitation and Gaining Access).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.6 (Generating your own Payloads).pdf...
Processing G:\7Th-Semester\information-Security\Pre-Mid\Handout 2.7 (Privilege Escalation

C:\Users\dell\AppData\Local\Temp\ipykernel_8228\4020492849.py:38: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist()


Enter your query:  what are the main concepts of os ?


Retrieving relevant documents...
Querying Llama 3.2 model...

Response from Llama 3.2:
The main concepts of an Operating System (OS) mentioned in the context are:

1. Convenience of user
2. Efficient operation of the computer system
